<a href="https://colab.research.google.com/github/mirpouya/TensorFlow-Tutorial/blob/main/Human_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"pouyammsadeghi","key":"773ac957be00991204924caaf65a4f47"}'}

In [4]:
 ! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                      title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
thedrcat/daigt-v2-train-dataset                          DAIGT V2 Train Dataset                           29MB  2023-11-16 01:38:36           2100        199  1.0              
muhammadbinimran/housing-price-prediction-data           Housing Price Prediction Data                   763KB  2023-11-21 17:56:32           9637        165  1.0              
thedrcat/daigt-proper-train-dataset                      DAIGT Proper Train Dataset                      119MB  2023-11-05 14:03:25           1909        155  1.0              
thedevastator/netflix-imdb-scores                        Netflix IMDB Scores                             699KB  202

In [9]:
! kaggle datasets download -d muhammadhananasghar/human-emotions-datasethes

 98% 303M/309M [00:01<00:00, 214MB/s]
100% 309M/309M [00:01<00:00, 164MB/s]


In [ ]:
! mkdir human-emotions-datasethes
! unzip human-emotions-datasethes.zip -d human-emotions-datasethes